In [2]:
import  os 
import  psycopg2  as  ps 
import  pandas  as  pd

conn = ps.connect(
    host=os.environ["db_host"],
    port=os.environ["db_port"],
    dbname=os.environ["db_name"],
    user=os.environ["db_user"],
    password=os.environ["db_password"])

##### Documentos JSON
Um documento(objeto) é um conjunto não-ordenado de dados armazenados em um par **"nome":valor** (campo), que inicia e termina com chaves {}.

- Todos os nomes-chaves são englobados em **aspas duplas**, e são sperados de seus valores por dois-pontos: 
- Os pares (campos) são separados um do outro por vírgulas.

- **"chave": chave**

###### Exemplo

{

    
    "chave": "string",
    "chave": 12,
    "chave": ["string",12.5,"string],
    "chave": true,
    "chave": {"chave":"valor"}
}

In [ ]:
--Criar tabela campo JSON
CREATE TABLE peds (
    id serial NOT NULL PRIMARY KEY,
    pedido json NOT NULL
);

--inserir registros para testes
INSERT INTO peds(pedidos)
VALUES
('{"comprador":"Syanne","produtos":{"bebida":"Suco de maracujá","comida":"Coxinha de frango"}}'),
('{"comprador":"Karoline","produtos":{"bebida":"Água","comida":"pizza de calabresa"}}'),
('{"comprador":"Moreira","produtos":{"bebida":"coca cola","comida":"Pastel de Frango"}}'),
('{"comprador":"Moreira","produtos":{"bebida":"guarana garoto","comida":"Salgado"}}')

-- Assim se pode consultar partes específicas de uma mesma coluna com uma complexidade maior
-- ùtil pra aplicações web em que se envia e recebe dados no formato JSON


In [3]:
df = pd.read_sql_query("select * from peds", con=conn)
df

C:\Users\syann\AppData\Local\Temp\ipykernel_9092\1205731766.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("select * from peds", con=conn)


,id,pedido
0,1,"{'comprador': 'Syanne', 'produtos': {'bebida':..."
1,2,"{'comprador': 'Karoline', 'produtos': {'bebida..."
2,3,"{'comprador': 'Moreira', 'produtos': {'bebida'..."
3,4,"{'comprador': 'Moreira', 'produtos': {'bebida'..."


#### Operadores JSON 
Operadores específicos pra acessar partes específicas da estrutura

In [ ]:
--Retornar nomes dos compradores no formato JSON
-- SELECT nome_coluna -> nome_chave FROM nome_tabela
-- "->" retorna um campo específico

SELECT pedido -> 'comprador' AS Comprador
FROM peds;

-- OBS : retorna no formato JSON 

In [4]:
df = pd.read_sql_query("SELECT pedido -> 'comprador' AS Comprador FROM peds", con=conn)
df

C:\Users\syann\AppData\Local\Temp\ipykernel_9092\3931384518.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT pedido -> 'comprador' AS Comprador FROM peds", con=conn)


,comprador
0,Syanne
1,Karoline
2,Moreira
3,Moreira


**Pra retornar em formato de texto: usar o operador *->>*.**

In [5]:
#idem anterior , mas em formato de texto
df = pd.read_sql_query("SELECT pedido ->> 'comprador' AS Comprador FROM peds", con=conn)
df

C:\Users\syann\AppData\Local\Temp\ipykernel_9092\1627589087.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT pedido ->> 'comprador' AS Comprador FROM peds", con=conn)


,comprador
0,Syanne
1,Karoline
2,Moreira
3,Moreira


In [10]:
#retornar as bebidas que foram pedidas
# SELECT pedido -> 'produtos' ->> 'bebida' AS Bebidas FROM peds

df = pd.read_sql_query("SELECT pedido -> 'produtos' ->> 'bebida' AS Bebidas FROM peds", con=conn)
df

C:\Users\syann\AppData\Local\Temp\ipykernel_9092\1951327678.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT pedido -> 'produtos' ->> 'bebida' AS Bebidas FROM peds", con=conn)


,bebidas
0,Suco de maracujá
1,Água
2,coca cola
3,guarana garoto


#### Json na cláusua WHERE: filtros

In [ ]:
--Quem comprou uma pizza de calabresa?
SELECT pedido -> 'comprador' AS Comprador
FROM peds
WHERE pedido -> 'produtos' ->> 'comida' = 'pizza de calabresa';

In [15]:
df = pd.read_sql_query("SELECT pedido -> 'comprador' AS Comprador FROM peds WHERE pedido -> 'produtos' ->> 'comida' = 'pizza de calabresa';", con=conn)
df

C:\Users\syann\AppData\Local\Temp\ipykernel_9092\2192696319.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT pedido -> 'comprador' AS Comprador FROM peds WHERE pedido -> 'produtos' ->> 'comida' = 'pizza de calabresa';", con=conn)


,comprador
0,Karoline


#### Funções JSON

- json_each
    -Retorna cada chave valor em uma linha separada
    - SELECT json_each(pedido) FROM peds;

In [16]:
df = pd.read_sql_query("SELECT json_each(pedido) FROM peds;", con=conn)
df

C:\Users\syann\AppData\Local\Temp\ipykernel_9092\1849449838.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT json_each(pedido) FROM peds;", con=conn)


,json_each
0,"(comprador,""""""Syanne"""""")"
1,"(produtos,""{""""bebida"""":""""Suco de maracujá"""",""""..."
2,"(comprador,""""""Karoline"""""")"
3,"(produtos,""{""""bebida"""":""""Água"""",""""comida"""":""""p..."
4,"(comprador,""""""Moreira"""""")"
5,"(produtos,""{""""bebida"""":""""coca cola"""",""""comida""..."
6,"(comprador,""""""Moreira"""""")"
7,"(produtos,""{""""bebida"""":""""guarana garoto"""",""""co..."


- Retornar em formato de texto
    - json_each_text
    

In [17]:
df = pd.read_sql_query("SELECT json_each_text(pedido) FROM peds;", con=conn)
df

C:\Users\syann\AppData\Local\Temp\ipykernel_9092\903461139.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT json_each_text(pedido) FROM peds;", con=conn)


,json_each_text
0,"(comprador,Syanne)"
1,"(produtos,""{""""bebida"""":""""Suco de maracujá"""",""""..."
2,"(comprador,Karoline)"
3,"(produtos,""{""""bebida"""":""""Água"""",""""comida"""":""""p..."
4,"(comprador,Moreira)"
5,"(produtos,""{""""bebida"""":""""coca cola"""",""""comida""..."
6,"(comprador,Moreira)"
7,"(produtos,""{""""bebida"""":""""guarana garoto"""",""""co..."


- json_object_keys
    - mostra as chaves usadas em uma coluna

In [19]:
df = pd.read_sql_query("SELECT json_object_keys(pedido->'produtos') FROM peds;", con=conn)
df

C:\Users\syann\AppData\Local\Temp\ipykernel_9092\894979693.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT json_object_keys(pedido->'produtos') FROM peds;", con=conn)


,json_object_keys
0,bebida
1,comida
2,bebida
3,comida
4,bebida
5,comida
6,bebida
7,comida


www.json.org
https://www.postgresql.org/docs/9.5/functions-json.html
